In [1]:
import pandas as pd
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
urlav = "https://raw.githubusercontent.com/olagit/Structure-vs-properties-of-compounds/main/data/maccs.csv"
maccs = pd.read_csv(urlav)

In [3]:
maccs = maccs.drop(["index", "smiles"], axis=1)
maccs

,logP,fp0,fp1,fp2,fp3,fp4,fp5,fp6,fp7,fp8,...,fp157,fp158,fp159,fp160,fp161,fp162,fp163,fp164,fp165,fp166
0,-0.770,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,1,1,1,1,0
1,-3.300,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
2,-2.060,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,-7.870,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
4,-1.330,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,-1.710,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1124,0.106,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,1,0,0
1125,-3.091,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,1,0,0
1126,-3.180,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [4]:
X = maccs.iloc[:,1:]
y = maccs.iloc[:,0]
X = torch.tensor(X.values)
Y = torch.tensor(y.values)
X.shape

torch.Size([1128, 167])

In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm.notebook import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

from typing import List, Tuple

def train(X_train, y_train, X_valid, y_valid):
    hidden_size = 512
    epochs = 50
    batch_size = 64
    learning_rate = 0.0001
    
    n_timesteps, n_features, n_outputs = X_train.shape[0], X_train.shape[1], y_train.shape[0]
    
    print(n_timesteps)
    print(n_features)
    print(n_outputs)
    
    model = nn.Sequential(
                          nn.Conv1d(n_timesteps, 64, 2),
                          nn.ReLU(),
                          nn.Conv1d(64, 64, 2),
                          nn.ReLU(),
                          torch.nn.Dropout(p=0.5),
                          nn.MaxPool1d(2),
                          nn.Flatten(),
                          nn.Linear(32*n_timesteps, 100),
                          nn.ReLU(),
                          nn.Linear(100, n_outputs),
                          nn.Softmax(dim=1))
                          
    model.train()

    dataset = TensorDataset(torch.FloatTensor(X_train.float()), torch.FloatTensor(y_train.float().reshape(-1, 1)))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    print("loader")

    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    loss_fn = nn.MSELoss()
    for epoch in trange(1, epochs + 1, leave=False):
        for X, y in tqdm(loader, leave=False):
            model.zero_grad()
            print("X")
            print(X.shape)
#             X = X.unsqueeze(1)
            print("unsqu")
            print(X.shape)
            output = model(X)
            loss = loss_fn(output,y)
            loss.backward()
            optimizer.step()
    return model


def predict(model, X_test, y_test):
    batch_size = 64

    dataset = TensorDataset(torch.FloatTensor(X_train.float()), torch.FloatTensor(y_train.float().reshape(-1, 1)))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    preds_batches = []
    with torch.no_grad():
        for X, Y in tqdm(loader):
            preds = model(X)
            preds_batches.append(preds.cpu().detach().numpy())
    preds = np.concatenate(preds_batches)
    return preds

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.7, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.5, random_state=123)
# print(torch.FloatTensor(y_test.float()))
# print(y_test.type())
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

scores = []

for _ in range(X_train.shape[0]):
    model = train(X_train, y_train, X_valid, y_valid)
    
    predictions = predict(model, X_test, y_test)
    
    rmse_score = np.sqrt(np.power(y_test - predictions, 2).mean())
    mae_score = np.abs(y_test - predictions).mean()
    scores.append([rmse_score, mae_score])
    
#     break
    
scores = np.array(scores)
print('RMSE, MAE = ' + \
      ', '.join(f'{mean:.2f}±{std:.3f}' for mean, std in zip(scores.mean(axis=0), scores.std(axis=0))))

print(predictions)


torch.Size([169, 167])
torch.Size([790, 167])
torch.Size([169])
torch.Size([790])
169
167
169
loader


X
torch.Size([64, 167])
unsqu
torch.Size([64, 167])


RuntimeError: Expected 3-dimensional input for 3-dimensional weight [64, 169, 2], but got 2-dimensional input of size [64, 167] instead